# Environment Setting

In [1]:
# https://github.com/NVIDIA/NeMo
# https://github.com/NVIDIA/NeMo-Megatron-Launcher
# https://github.com/NVIDIA/Megatron-LM

# conda create -n py310 python=3.10
# # sudo apt update
# # sudo apt install libsndfile1 ffmpeg
# conda activate py310

# pip install -U Cython
# pip install -U python-dotenv

# pip install -U nemo_toolkit['all']

# pip install -U -v --no-build-isolation --disable-pip-version-check --no-cache-dir --config-settings \
# "--build-option=--cpp_ext --cuda_ext --fast_layer_norm --distributed_adam --deprecated_fused_adam" git+https://github.com/NVIDIA/apex.git@c07a4cf67102b9cd3f97d1ba36690f985bae4227

# pip install -U git+https://github.com/NVIDIA/NeMo-Megatron-Launcher.git
# pip install -U git+https://github.com/NVIDIA/Megatron-LM.git
# pip install -U git+https://github.com/NVIDIA/NeMo.git

# Dataset Preparation

In [16]:
! git clone https://huggingface.co/datasets/databricks/databricks-dolly-15k
# ! wget https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

Cloning into 'databricks-dolly-15k'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 36 (delta 7), reused 36 (delta 7), pack-reused 0
Unpacking objects: 100% (36/36), 8.39 KiB | 1.40 MiB/s, done.


In [17]:
! du -sh databricks-dolly-15k/databricks-dolly-15k.jsonl

13M	databricks-dolly-15k/databricks-dolly-15k.jsonl


In [18]:
! head -5 databricks-dolly-15k/databricks-dolly-15k.jsonl

{"instruction": "When did Virgin Australia start operating?", "context": "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", "response": "Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.", "category": "closed_qa"}
{"instruction": "Which is a species of fish? Tope or Rope", "context": "", "response": "Tope", "category": "classification"}
{"instruction": "Why can camels survive for long without water?", "context": "", "response": "Camels use the fat in their humps to keep them 

In [19]:
# ! python NeMo-Megatron-Launcher/launcher_scripts/nemo_launcher/collections/dataprep_scripts/dolly_dataprep/preprocess.py \
#     --input databricks-dolly-15k/databricks-dolly-15k.jsonl

Preprocessing data to jsonl format...
Data was successfully preprocessed and saved by databricks-dolly-15k/databricks-dolly-15k-output.jsonl .


In [20]:
! head -5 databricks-dolly-15k/databricks-dolly-15k-output.jsonl

{"input": "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.\n\nWhen did Virgin Australia start operating?", "output": "Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.", "category": "closed_qa"}
{"input": "Which is a species of fish? Tope or Rope", "output": "Tope", "category": "classification"}
{"input": "Why can camels survive for long without water?", "output": "Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.", "cat

# Base model preparation

In [7]:
import os
from huggingface_hub import snapshot_download
from dotenv import load_dotenv
load_dotenv() # HF_TOKEN
HF_TOKEN = os.environ["HF_TOKEN"]

In [8]:
os.makedirs("gpt-2b-001-hf", exist_ok=True)

In [9]:
snapshot_download(
    repo_id="nvidia/GPT-2B-001", 
    local_dir="gpt-2b-001-hf", 
    local_dir_use_symlinks=False, 
    token=HF_TOKEN
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

GPT-2B-001_bf16_tp1.nemo:   0%|          | 0.00/9.02G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

'/home/quantm/NeMo_notebooks/gpt-2b-001-hf'

In [22]:
# %%writefile split_train_val.py
import json
import random

data_dir = "databricks-dolly-15k"
input_file = f"{data_dir}/databricks-dolly-15k-output.jsonl"
training_output_file = f"{data_dir}/training.jsonl"
validation_output_file = f"{data_dir}/validation.jsonl"
test_output_file = f"{data_dir}/test.jsonl"

# Specify the proportion of data for training and validation
train_proportion = 0.80
validation_proportion = 0.15
test_proportion = 0.05

# Read the JSONL file and shuffle the JSON objects
with open(input_file, "r") as f:
    lines = f.readlines()
    random.shuffle(lines)

# Calculate split indices
total_lines = len(lines)
train_index = int(total_lines * train_proportion)
val_index = int(total_lines * validation_proportion)

# Distribute JSON objects into training and validation sets
train_data = lines[:train_index]
validation_data = lines[train_index:train_index+val_index]
test_data = lines[train_index+val_index:]

# # Write JSON objects to training file
# with open(training_output_file, "w") as f:
#     for line in train_data:
#         f.write(line.strip() + "\n")

# # Write JSON objects to validation file
# with open(validation_output_file, "w") as f:
#     for line in validation_data:
#         f.write(line.strip() + "\n")

# # Write JSON objects to training file
# with open(test_output_file, "w") as f:
#     for line in test_data:
#         f.write(line.strip() + "\n")

In [23]:
# ! python split_train_val.py

# Set the Configuration

In [24]:
# Import the dotenv module
from dotenv import dotenv_values

# Define the variables
data_dir = "databricks-dolly-15k"
TRAIN = f"[{data_dir}/training.jsonl]"
VALID = f"[{data_dir}/validation.jsonl]"
TEST = f"[{data_dir}/test.jsonl]"
VALID_NAMES = f"[databricks-dolly-15k]"

CONCAT_SAMPLING_PROBS="[1.0]"
TP_SIZE=1
PP_SIZE=1

# Create a dictionary with the variables
variables = {
    "TRAIN": TRAIN,
    "VALID": VALID,
    "TEST": TEST,
    "VALID_NAMES": VALID_NAMES,
    "CONCAT_SAMPLING_PROBS": CONCAT_SAMPLING_PROBS, 
    "TP_SIZE": TP_SIZE, 
    "PP_SIZE": PP_SIZE, 
}

# Write the variables to the .env file
with open('.env', 'a') as f:
    for key, value in variables.items():
        f.write(f"{key}={value}\n")

# Read back the values from the .env file to verify
config = dotenv_values(".env")
# display(config)

# Training

In [25]:
command = '''\
dotenv -f .env run \
python NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
    trainer.precision=bf16 \
    trainer.max_steps=50 \
    trainer.devices=1 \
    trainer.val_check_interval=0.1 \
    model.megatron_amp_O2=True \
    model.restore_from_path=gpt-2b-001-hf/GPT-2B-001_bf16_tp1.nemo \
    model.tensor_model_parallel_size=$TP_SIZE \
    model.pipeline_model_parallel_size=$PP_SIZE \
    model.optim.lr=5e-6 \
    model.answer_only_loss=True \
    model.data.train_ds.micro_batch_size=1 \
    model.data.train_ds.global_batch_size=128 \
    model.data.train_ds.file_names=$TRAIN \
    model.data.train_ds.concat_sampling_probabilities=$CONCAT_SAMPLING_PROBS \
    model.data.validation_ds.micro_batch_size=1 \
    model.data.validation_ds.global_batch_size=128 \
    model.data.validation_ds.file_names=$VALID \
    model.data.validation_ds.names=$VALID_NAMES \
    model.data.test_ds.file_names=$TEST \
    model.data.test_ds.micro_batch_size=1 \
    model.data.test_ds.global_batch_size=256 \
    model.data.train_ds.num_workers=0 \
    model.data.validation_ds.num_workers=0 \
    model.data.test_ds.num_workers=0 \
    model.data.validation_ds.metric.name=loss \
    model.data.test_ds.metric.name=loss \
    exp_manager.create_wandb_logger=False \
    exp_manager.explicit_log_dir=results \
    exp_manager.resume_if_exists=True \
    exp_manager.resume_ignore_no_checkpoint=True \
    exp_manager.create_checkpoint_callback=True \
    exp_manager.checkpoint_callback_params.monitor=validation_loss
'''

os.system(command)

[NeMo W 2024-02-01 15:30:42 nemo_logging:349] /home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo W 2024-02-01 15:30:42 nemo_logging:349] /home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/lightning_fabric/connector.py:554: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2024-02-01 15:30:42 exp_manager:694] Exp_manager is logging to results, but it already exists.
[NeMo W 2024-02-01 15:30:42 exp_manager:616] There were no checkpoints

[NeMo I 2024-02-01 15:30:42 megatron_gpt_finetuning:56] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-02-01 15:30:42 megatron_gpt_finetuning:57] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: bf16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 50
      log_every_n_steps: 10
      val_check_interval: 0.1
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: results
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_loss
        save_top_k: 1
        mode: min
        save_nemo_on_train_end: true
       

[NeMo W 2024-02-01 15:30:50 megatron_gpt_model:238] megatron-core was not found. Please see the NeMo README for installation instructions: https://github.com/NVIDIA/NeMo#megatron-gpt.
[NeMo W 2024-02-01 15:30:50 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-02-01 15:30:50 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-02-01 15:30:50 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: tp_comm_overlap in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-02-01 15:30:50 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: tp_comm_split_ag in its cfg. Add this key to cfg o

[NeMo I 2024-02-01 15:30:50 megatron_init:241] Rank 0 has data parallel group : [0]
[NeMo I 2024-02-01 15:30:50 megatron_init:247] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2024-02-01 15:30:50 megatron_init:252] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2024-02-01 15:30:50 megatron_init:255] Ranks 0 has data parallel rank: 0
[NeMo I 2024-02-01 15:30:50 megatron_init:272] Rank 0 has context parallel group: [0]
[NeMo I 2024-02-01 15:30:50 megatron_init:275] All context parallel group ranks: [[0]]
[NeMo I 2024-02-01 15:30:50 megatron_init:276] Ranks 0 has context parallel rank: 0
[NeMo I 2024-02-01 15:30:50 megatron_init:287] Rank 0 has model parallel group: [0]
[NeMo I 2024-02-01 15:30:50 megatron_init:288] All model parallel group ranks: [[0]]
[NeMo I 2024-02-01 15:30:50 megatron_init:298] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-02-01 15:30:50 megatron_init:302] All tensor model parallel group ranks: 

[NeMo W 2024-02-01 15:30:57 nemo_logging:349] /home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/modules/module.py:1877: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
      warnings.warn(
    


...

[NeMo I 2024-02-01 16:08:35 nlp_overrides:448] Removing checkpoint: results/checkpoints/megatron_gpt_peft_adapter_tuning--validation_loss=2.192-step=40-consumed_samples=5120.0-last.ckpt
Epoch 0: : 100%|██████████| 50/50 [38:31<00:00, v_num=, reduced_train_loss=2.210, global_step=49.00, consumed_samples=6400.0, train_step_timing in s=15.80, val_loss=2.150]
Validation: 0it [00:00, ?it/s]
Epoch 0: : 100%|██████████| 50/50 [41:19<00:00, v_num=, reduced_train_loss=2.210, global_step=49.00, consumed_samples=6400.0, train_step_timing in s=15.80, val_loss=2.110]
                                                  [NeMo I 2024-02-01 16:12:42 nlp_overrides:448] Removing checkpoint: results/checkpoints/megatron_gpt_peft_adapter_tuning--validation_loss=2.151-step=45-consumed_samples=5760.0.ckpt


Metric val_loss improved by 0.044 >= min_delta = 0.001. New best score: 2.106
Epoch 0, global step 50: 'validation_loss' reached 2.10617 (best 2.10617), saving model to 'results/checkpoints/megatron_gpt_peft_adapter_tuning--validation_loss=2.106-step=50-consumed_samples=6400.0.ckpt' as top 1


[NeMo I 2024-02-01 16:12:42 nlp_overrides:448] Removing checkpoint: results/checkpoints/megatron_gpt_peft_adapter_tuning--validation_loss=2.151-step=45-consumed_samples=5760.0-last.ckpt
Epoch 0: : 100%|██████████| 50/50 [41:20<00:00, v_num=, reduced_train_loss=2.210, global_step=49.00, consumed_samples=6400.0, train_step_timing in s=15.80, val_loss=2.110]


`Trainer.fit` stopped: `max_steps=50` reached.
Restoring states from the checkpoint path at results/checkpoints/megatron_gpt_peft_adapter_tuning--validation_loss=2.106-step=50-consumed_samples=6400.0.ckpt
Restored all states from the checkpoint at results/checkpoints/megatron_gpt_peft_adapter_tuning--validation_loss=2.106-step=50-consumed_samples=6400.0.ckpt


0

In [26]:
command = '''\
! dotenv -f .env run \
python NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py \
    model.restore_from_path=results/checkpoints/megatron_gpt_peft_adapter_tuning.nemo \
    model.peft.restore_from_path=null \
    trainer.devices=1 \
    model.data.test_ds.file_names=['dolly-15k_test.jsonl'] \
    model.data.test_ds.names=['dolly-15k_test.jsonl'] \
    model.data.test_ds.global_batch_size=4 \
    model.data.test_ds.micro_batch_size=4 \
    model.data.test_ds.tokens_to_generate=20 \
    inference.greedy=True \
    inference.outfile_path='results/sft_results.jsonl'  
'''

os.system(command)

[NeMo W 2024-02-01 16:13:03 nemo_logging:349] /home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo W 2024-02-01 16:13:03 nemo_logging:349] /home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2024-02-01 16:13:03 megatron_gpt_model:238] megatron-core was not found. Please see the NeMo README for installation instructions: https://github.com/NVIDIA/NeMo#megatro

[NeMo I 2024-02-01 16:13:03 megatron_gpt_generate:114] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-02-01 16:13:03 megatron_gpt_generate:115] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 20000
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.test_ds.metric.name}
        save_top_k: 1
        mode: max
        save_nemo_o

[NeMo W 2024-02-01 16:13:03 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-02-01 16:13:03 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-02-01 16:13:03 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: tp_comm_overlap in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-02-01 16:13:03 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: tp_comm_split_ag in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-02-01 16:13:03 megatron_base_model:1078] The model: MegatronGPTSFTModel() does not have field.name: tp_comm_atomic_ag in 

[NeMo I 2024-02-01 16:13:03 megatron_init:241] Rank 0 has data parallel group : [0]
[NeMo I 2024-02-01 16:13:03 megatron_init:247] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2024-02-01 16:13:03 megatron_init:252] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2024-02-01 16:13:03 megatron_init:255] Ranks 0 has data parallel rank: 0
[NeMo I 2024-02-01 16:13:03 megatron_init:272] Rank 0 has context parallel group: [0]
[NeMo I 2024-02-01 16:13:03 megatron_init:275] All context parallel group ranks: [[0]]
[NeMo I 2024-02-01 16:13:03 megatron_init:276] Ranks 0 has context parallel rank: 0
[NeMo I 2024-02-01 16:13:03 megatron_init:287] Rank 0 has model parallel group: [0]
[NeMo I 2024-02-01 16:13:03 megatron_init:288] All model parallel group ranks: [[0]]
[NeMo I 2024-02-01 16:13:03 megatron_init:298] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-02-01 16:13:03 megatron_init:302] All tensor model parallel group ranks: 

Error executing job with overrides: ['model.restore_from_path=results/checkpoints/megatron_gpt_peft_adapter_tuning.nemo', 'model.peft.restore_from_path=null', 'trainer.devices=1', 'model.data.test_ds.file_names=[dolly-15k_test.jsonl]', 'model.data.test_ds.names=[dolly-15k_test.jsonl]', 'model.data.test_ds.global_batch_size=4', 'model.data.test_ds.micro_batch_size=4', 'model.data.test_ds.tokens_to_generate=20', 'inference.greedy=True', 'inference.outfile_path=results/sft_results.jsonl']
Traceback (most recent call last):
  File "/home/quantm/NeMo_notebooks/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py", line 123, in main
    model = MegatronGPTSFTModel.restore_from(cfg.model.restore_from_path, model_cfg, trainer=trainer)
  File "/home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/nemo/collections/nlp/models/nlp_model.py", line 465, in restore_from
    return super().restore_from(
  File "/home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/nemo

0

In [27]:
! tail -n 5 results/sft_results.jsonl

tail: cannot open 'results/sft_results.jsonl' for reading: No such file or directory
